# 1 Import necessary packages and libraries

In [ ]:
# Importing necessary packages
import requests
from zipfile import ZipFile
import pandas as pd
from io import BytesIO

In [ ]:
def read_dataframe_from_url(url, **kwargs):
    """
    Function to download and read a CSV file from a URL, directly supporting CSV files
    and zip files containing CSV files. Accepts additional arguments for the pd.read_csv method.

    Parameters:
    - url (str): URL of the file to be read.
    - **kwargs: Additional arguments to be passed to pd.read_csv.

    Returns:
    - pd.DataFrame: DataFrame containing the data from the CSV file.
    - None: In case of download failure or error reading the file.

    Usage example:
    >>> url = "https://example.com/file.zip"
    >>> df = read_dataframe_from_url(url, sep=';')  # Additional arguments can be specified here
    >>> if df is not None:
    ...     print(df.head())
    ... else:
    ...     print("Failed to read the DataFrame.")
    """
    try:
        # Download the file from the URL
        response = requests.get(url)

        # Check if the download was successful (status code 200)
        if response.status_code == 200:
            # Check the content type (zip or csv)
            if response.headers['content-type'] == 'application/zip':
                # Extract the contents of the zip file
                with ZipFile(BytesIO(response.content)) as zip_file:
                    # Assuming there is only one .csv file in the zip, you can get the first in the list
                    csv_file_name = zip_file.namelist()[0]

                    # Read the CSV file into a pandas DataFrame
                    with zip_file.open(csv_file_name) as csv_file:
                        df = pd.read_csv(csv_file, **kwargs)
            elif response.headers['content-type'] == 'text/csv':
                # If it's a direct CSV file, read it into a pandas DataFrame
                df = pd.read_csv(BytesIO(response.content), **kwargs)
            else:
                raise ValueError(f"Unsupported file format for URL: {url}")

            return df

        else:
            raise requests.exceptions.RequestException(f"Download failed. Status code: {response.status_code} for URL: {url}")

    except Exception as e:
        print(f"Error during function execution: {e}")
        return None


In [ ]:
def read_dataframe_from_url_with_log(url, **kwargs):
    try:
        df = read_dataframe_from_url(url, **kwargs)
        if df is not None:
            print(f"Successful read for {url}")
            return df
        else:
            print(f"Failed to read for {url}")
            return None
    except Exception as e:
        print(f"Error during reading {url}: {e}")
        return None

def build_anp_ca_base(url_paths, sep=';', decimal=',', encoding='latin-1', **kwargs):
    """
    Function to build the historical base of automotive fuels from ANP available on the dados.gov data portal.

    Parameters:
    - url_paths (list): List of URLs to download the semiannual histories.
    - sep (str): CSV file delimiter.
    - decimal (str): Decimal separator in the CSV file.
    - encoding (str): Encoding to be used when reading the CSV file.
    - **kwargs: Additional arguments to be passed to the read_dataframe_from_url function.

    Returns:
    - pd.DataFrame: DataFrame compiling the historical bases.
    - None: In case of download failure or error reading the files.
    """
    try:
        dfs = []

        for url in url_paths:
            df_aux = read_dataframe_from_url_with_log(url, sep=sep, decimal=decimal, encoding=encoding, **kwargs)
            if df_aux is not None:
                df_aux.columns = [
                    'Regiao_Sigla', 'Estado_Sigla', 'Municipio', 'Revenda',
                    'CNPJ_da_Revenda', 'Nome_da_Rua', 'Numero_Rua', 'Complemento', 'Bairro',
                    'Cep', 'Produto', 'Data_da_Coleta', 'Valor_de_Venda', 'Valor_de_Compra',
                    'Unidade_de_Medida', 'Bandeira'
                ]
                dfs.append(df_aux)
            else:
                print(f"Failed to read the path: {url}")
                return None

        df = pd.concat(dfs, ignore_index=True)
        print("Successful concatenation.")
        return df

    except Exception as e:
        print(f"Error during function execution: {e}")
        return None

# 2 Data Reading

In [ ]:
# Description of the database as well as the data and their respective metadata
# available at https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/serie-historica-de-precos-de-combustiveis

path_dados_gov = 'https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/' # download link
url_paths = [
    path_dados_gov + 'ca-2019-02.csv',
    path_dados_gov + 'ca-2020-01.csv',
    path_dados_gov + 'ca-2020-02.csv',
    path_dados_gov + 'ca-2021-01.csv',
    path_dados_gov + 'ca-2021-02.csv',
    path_dados_gov + 'precos-semestrais-ca.zip',
    path_dados_gov + 'ca-2022-02.zip',
    path_dados_gov + 'ca-2023-01.zip',
    path_dados_gov + 'ca-2023-02.zip',
]

In [ ]:
df_anp_ca = build_anp_ca_base(url_paths, sep=';', decimal=',', encoding='latin-1')

Leitura bem-sucedida para https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2019-02.csv
Leitura bem-sucedida para https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2020-01.csv
Leitura bem-sucedida para https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2020-02.csv
Leitura bem-sucedida para https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2021-01.csv
Leitura bem-sucedida para https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2021-02.csv
Leitura bem-sucedida para https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/precos-semestrais-ca.zip
Leitura bem-sucedida para https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2022-02.zip
Leitura bem-sucedida para https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/sh

In [ ]:
if df_anp_ca is not None:
    df_anp_ca.info()
else:
    print("Failed to load the base!!")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3845638 entries, 0 to 3845637
Data columns (total 16 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Regiao_Sigla       object 
 1   Estado_Sigla       object 
 2   Municipio          object 
 3   Revenda            object 
 4   CNPJ_da_Revenda    object 
 5   Nome_da_Rua        object 
 6   Numero_Rua         object 
 7   Complemento        object 
 8   Bairro             object 
 9   Cep                object 
 10  Produto            object 
 11  Data_da_Coleta     object 
 12  Valor_de_Venda     float64
 13  Valor_de_Compra    float64
 14  Unidade_de_Medida  object 
 15  Bandeira           object 
dtypes: float64(2), object(14)
memory usage: 469.4+ MB


In [ ]:
# saving the sample
df_anp_ca.to_parquet('df.parquet.gzip',
              compression='gzip')